In [3]:
print('Takumi House & Kailen Santos')

Takumi House & Kailen Santos


In [4]:
from matplotlib import pyplot as plt
import matplotlib as mp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.manifold
from scipy import cluster
import seaborn as sns
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import requests
import json
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import re
import json
import pickle
%matplotlib inline

In [5]:
##load in pickled dataframe of all the reviews

#try:
#    with open('df_reviews','rb') as f:
#        df_reviews = pickle.load(f)
        
#except FileNotFoundError:
#    with open('df_reviews','wb') as f:
#        pickle.dump(df_reviews, f)

KeyboardInterrupt: 

In [6]:
##load in pickled stemmed data for the review text
try:
    with open('stemmed_data','rb') as f:
        stemmed_data = pickle.load(f)
        
except FileNotFoundError:
    stemmed_data = [" ".join(SnowballStemmer("english", ignore_stopwords=True).stem(word)  
        for sent in sent_tokenize(message)
        for word in word_tokenize(sent))
        for message in df_reviews['text']]
    
    with open('stemmed_data','wb') as f:
        pickle.dump(stemmed_data, f)

In [7]:
df = pd.read_csv('Restaurant_Inspections.csv',header=0)
# df = df[df.City == 'Las Vegas']
# df = (df[df['Inspection Result'] == 'Closed with Fees']).append(df[df['Inspection Result'] == 'Closed without Fees'])

df.shape

#with open ('Resaurant_Inspections.csv') as file:
#   for line in file:
#      if (line[6]=='Las Vegas' && line[]

C:\Users\Kailen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(123919, 23)

In [8]:
df = df[df.City == 'Las Vegas']
df.shape

(94700, 23)

In [9]:
categories = df['Category Name'].unique().tolist()
print(categories)

['Pantry', 'Special Kitchen', 'Garde Manger', 'Kitchen Bakery', 'Bar / Tavern', 'Buffet', 'Snack Bar', 'Restaurant', 'Banquet Support', 'Portable Unit', 'Banquet Kitchen', 'Portable Bar', 'Meat/Poultry/Seafood', 'Vegetable Prep', 'Caterer', 'Barbeque', 'Confection', 'Concessions', 'Produce Market', 'Institutional Food Service', 'Elementary School Kitchen', 'Self-Service Food Truck', 'Bakery Sales', 'Food Trucks / Mobile Vendor', 'Grocery Store Sampling', 'Beer Bar', 'Childcare Kitchens', "Farmer's Market", 'Main Kitchen', 'Frozen Meat Sales']


In [10]:
categories.remove('Elementary School Kitchen')
categories.remove('Grocery Store Sampling')
categories.remove('Farmer\'s Market')
categories.remove('Caterer')
categories.remove('Concessions')
categories.remove('Institutional Food Service')
categories.remove('Banquet Support')

In [11]:
df = df[df['Category Name'].isin(categories)]
df.shape

(91735, 23)

In [12]:
coord_strings = df['Location 1'].tolist()

In [13]:
import ast
xcoords = []
ycoords = []
for coord in coord_strings:
    xcoords.append(ast.literal_eval(coord)[0])
    ycoords.append(ast.literal_eval(coord)[1])

In [14]:
count = 0
for idx, ycoord in enumerate(ycoords):
    if ycoord > 0:
        ycoords[idx] = ycoord * -1
        count += 1
print('{} coordinates fixed.'.format(count))

574 coordinates fixed.


In [15]:
df['Coordinates'] = list(zip(xcoords, ycoords))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates
11,DA0001715,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
12,DA0001716,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000567,Routine Inspection,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
13,DA0001728,PR0000427,CAESARS DIAMOND LOUNGE KITCHEN,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,8,...,EE7000417,Routine Inspection,4,A,NaN,Compliant,14,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
14,DA0001773,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,7,A,NaN,Compliant,14232831,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
15,DA0001776,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000392,Routine Inspection,2,A,NaN,Compliant,NaN,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"


In [16]:
coordinates_list = df['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])
df['X'] = xcoords
df['Y'] = ycoords

In [17]:
df = df[df['Y'] < -114.5]
df = df[df['Y'] > -115.6]
df_clean = df.reset_index(drop = True)

In [18]:
coordinates_list = df_clean['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])

In [19]:
df = df_clean[df_clean['Date Current']==df_clean['Inspection Date']]
df

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
63,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
196,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
254,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
317,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
348,DA001XLIF,PR0022088,Marketon #1 - Deli,Marketon #1,Snack Bar,840 N Decatur Blvd,Las Vegas,Nevada,89107-1905,8,...,8,A,NaN,'A' Grade,209217,07/11/2016 03:47:09 PM,"(36.180332, -115.205031)","(36.180332, -115.205031)",36.180332,-115.205031
436,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106
503,DA0025038,PR0001066,Kentucky Fried Chicken #527621,Kentucky Fried Chicken #X527621,Restaurant,4065 S Buffalo Dr,Las Vegas,Nevada,89147-7455,7,...,7,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1150528, -115.2625292)","(36.1150528, -115.2625292)",36.115053,-115.262529
505,DA0025516,PR0001062,Kentucky Fried Chicken / Pizza Hut,Kentucky Fried Chicken/Pizza Hut #X527066,Restaurant,5587 S Rainbow Blvd,Las Vegas,Nevada,89118-1857,5,...,5,A,NaN,Compliant,1420,02/21/2013 10:26:12 PM,"(36.0877539, -115.243508)","(36.0877539, -115.243508)",36.087754,-115.243508
531,DA0027055,PR0001064,Kentucky Fried Chicken #X527030,Kentucky Fried Chicken,Restaurant,9360 W Sahara Ave,Las Vegas,Nevada,89117-5351,2,...,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.144667, -115.299057)","(36.144667, -115.299057)",36.144667,-115.299057
533,DA0027285,PR0001068,Kentucky Fried Chicken #527019,Kentucky Fried Chicken #X527019,Restaurant,2490 E Desert Inn Rd,Las Vegas,Nevada,89121-3617,10,...,10,A,NaN,Compliant,1314203136,02/21/2013 10:26:12 PM,"(36.1301516, -115.116572)","(36.1301516, -115.116572)",36.130152,-115.116572


In [20]:
df_clean = df.reset_index(drop=True)

In [21]:
df_agg = df_clean.groupby('Restaurant Name',as_index=False).agg(lambda x: x.tolist())

In [22]:
dfcategories = df_agg.columns
dfcategories

Index(['Restaurant Name', 'Serial Number', 'Permit Number', 'Location Name',
       'Category Name', 'Address', 'City', 'State', 'Zip', 'Current Demerits',
       'Current Grade', 'Date Current', 'Inspection Date', 'Inspection Time',
       'Employee ID', 'Inspection Type', 'Inspection Demerits',
       'Inspection Grade', 'Permit Status', 'Inspection Result', 'Violations',
       'Record Updated', 'Location 1', 'Coordinates', 'X', 'Y'],
      dtype='object')

In [23]:
# drop unnecessary columns for categorization step later
df_agg = df_agg.drop(['Serial Number'], axis = 1)
df_agg = df_agg.drop(['City'], axis = 1)
df_agg = df_agg.drop(['State'], axis = 1)
df_agg = df_agg.drop(['Zip'], axis = 1)
df_agg = df_agg.drop(['Inspection Time'], axis = 1)
df_agg = df_agg.drop(['Employee ID'], axis = 1)
df_agg = df_agg.drop(['Inspection Type'], axis = 1)
df_agg = df_agg.drop(['Location 1'], axis = 1)
df_agg = df_agg.drop(['Coordinates'], axis = 1)
df_agg = df_agg.drop(['Record Updated'], axis = 1)
df_agg = df_agg.drop(['Inspection Date'], axis = 1)

In [24]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,[PR0110047],[CHINA DRAGON/#1 HAWAIIAN BARBEQUE],[Restaurant],[5905 S EASTERN Ave 105],[9],[A],[12/21/2015 12:00:00 AM],[9],[A],[nan],[Compliant],[211215292829302955],[36.0820114],[-115.119644]
1,$5 CAFE,[PR0017837],[AZTEC INN - $5 CAFE],[Restaurant],[2200 S Las Vegas Blvd],[10],[A],[01/07/2016 12:00:00 AM],[10],[A],[nan],[Compliant],[202206291029282930],[36.1459136],[-115.1561175]
2,100 DEGREES HOT POT - DELETED 10/15,[PR0113718],[100 DEGREES HOT POT],[Restaurant],[5600 SPRING MOUNTAIN Rd B],[3],[A],[04/27/2015 12:00:00 AM],[3],[A],[nan],[Compliant],[2122911],[36.1265084],[-115.2182851]
3,101 MART SNACK BAR,[PR0118514],[101 MART],[Snack Bar],[1021 CONVENTION CENTER Dr],[6],[A],[07/26/2016 12:00:00 AM],[6],[A],[nan],['A' Grade],[2152955],[36.1322288],[-115.1553506]
4,168 Market - Bakery,[PR0021447],[168 Market],[Bakery Sales],[3459 S Jones Blvd],[3],[A],[07/05/2016 12:00:00 AM],[3],[A],[nan],['A' Grade],[217],[36.1265813],[-115.2250088]
5,168 Market - Restaurant,[PR0021446],[168 Market],[Restaurant],[3459 S Jones Blvd],[0],[A],[10/14/2015 12:00:00 AM],[0],[A],[nan],[Compliant],[291029282930],[36.1265813],[-115.2250088]
6,168 Market / Produce Department,[PR0021445],[168 Market],[Produce Market],[3459 S Jones Blvd],[6],[A],[07/28/2016 12:00:00 AM],[6],[A],[nan],['A' Grade],[2152955],[36.1265813],[-115.2250088]
7,17 SOUTH BOOZE & BITES BAR,[PR0109450],[Tahiti Village],[Bar / Tavern],[7200 S Las Vegas Blvd],[0],[A],[05/18/2016 12:00:00 AM],[0],[A],[nan],[Compliant],[nan],[36.0596893],[-115.1715274]
8,17 SOUTH BOOZE & BITES GRILL,[PR0109449],[Tahiti Village],[Restaurant],[7200 S Las Vegas Blvd],[9],[A],[05/18/2016 12:00:00 AM],[9],[A],[nan],[Compliant],[21221629292955],[36.0596893],[-115.1715274]
9,1900 ASIAN CUISINE,[PR0099062],[1900 ASIAN CUISINE],[Restaurant],[5115 Spring Mountain Rd 103],[0],[A],[11/22/2013 12:00:00 AM],[0],[A],[nan],[Compliant],[29102927],[36.1255374],[-115.2112409]


In [25]:
dfcategories = df_agg.columns
for x in dfcategories[1:]:
    for y in range(len(df_agg[x])):
        df_agg[x][y] = list(set(df_agg[x][y]))
        df_agg[x][y] = df_agg[x][y][0]

In [26]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,PR0110047,CHINA DRAGON/#1 HAWAIIAN BARBEQUE,Restaurant,5905 S EASTERN Ave 105,9,A,12/21/2015 12:00:00 AM,9,A,NaN,Compliant,211215292829302955,36.082,-115.12
1,$5 CAFE,PR0017837,AZTEC INN - $5 CAFE,Restaurant,2200 S Las Vegas Blvd,10,A,01/07/2016 12:00:00 AM,10,A,NaN,Compliant,202206291029282930,36.1459,-115.156
2,100 DEGREES HOT POT - DELETED 10/15,PR0113718,100 DEGREES HOT POT,Restaurant,5600 SPRING MOUNTAIN Rd B,3,A,04/27/2015 12:00:00 AM,3,A,NaN,Compliant,2122911,36.1265,-115.218
3,101 MART SNACK BAR,PR0118514,101 MART,Snack Bar,1021 CONVENTION CENTER Dr,6,A,07/26/2016 12:00:00 AM,6,A,NaN,'A' Grade,2152955,36.1322,-115.155
4,168 Market - Bakery,PR0021447,168 Market,Bakery Sales,3459 S Jones Blvd,3,A,07/05/2016 12:00:00 AM,3,A,NaN,'A' Grade,217,36.1266,-115.225
5,168 Market - Restaurant,PR0021446,168 Market,Restaurant,3459 S Jones Blvd,0,A,10/14/2015 12:00:00 AM,0,A,NaN,Compliant,291029282930,36.1266,-115.225
6,168 Market / Produce Department,PR0021445,168 Market,Produce Market,3459 S Jones Blvd,6,A,07/28/2016 12:00:00 AM,6,A,NaN,'A' Grade,2152955,36.1266,-115.225
7,17 SOUTH BOOZE & BITES BAR,PR0109450,Tahiti Village,Bar / Tavern,7200 S Las Vegas Blvd,0,A,05/18/2016 12:00:00 AM,0,A,NaN,Compliant,NaN,36.0597,-115.172
8,17 SOUTH BOOZE & BITES GRILL,PR0109449,Tahiti Village,Restaurant,7200 S Las Vegas Blvd,9,A,05/18/2016 12:00:00 AM,9,A,NaN,Compliant,21221629292955,36.0597,-115.172
9,1900 ASIAN CUISINE,PR0099062,1900 ASIAN CUISINE,Restaurant,5115 Spring Mountain Rd 103,0,A,11/22/2013 12:00:00 AM,0,A,NaN,Compliant,29102927,36.1255,-115.211


In [28]:
try:
    with open('df_agg_revs','rb') as f:
        df_agg_revs = pickle.load(f)
        
except FileNotFoundError:
    with open('df_agg_revs','wb') as f:
        pickle.dump(non_0_sick_scores, f)

In [37]:
sick_ids = []
for x in df_agg_revs['business_id'].items():
    sick_ids.append(x[1])
print(sick_ids)

['0tUlPATGNNR7zq5uzh3nog', '9RjV5ZT6Np-VsK96_K1KlQ', '994TtJYxB68eSscnCr94qQ', 'fAJYeE4Q6vwVPGkk1MWCoQ', '2mKC1oXdGCl2w8SAZK8fjw', '9xcOsHNgiLSr18ZmFiU9Qg', 'XqmAbDasWtBd76LpJnMKqQ', 'r164gtGFkTMUqOmZXy2JtQ', 'g2ZRZXEKxj0SL86etbydcg', '3tiST9R0JRLbi4hw4RUBjA', 'wMRx5hvL9_739HYiFLw6Hw', 'vZ2dPk0fPQu4wrXx-DKeDw', 'ETnU0pf3aFuZfpHEHWyd4A', 'gIGZ8QFPlUaxVDTuz3oB9w', 'tc1w6LZUUB9gRMmKYzkIjQ', 'S10V82tt-8EzPF3Y6O7ZbA', 'FTQ1hm8-24_7ou2ILWEyEw', 'JM6CdlbuxfalZZ77DAq7Bg', 'GfuApsBX3jHdZGgWi8PTag', 'bTaIdTtPQIVaj7GChPXGBA', '94HshaJUV3DLvXkvbCbZqg', '0_1am1mjy0hJFyvpsk3x5g', 'LsArM1PhbqvXK3lDK-Ch_w', 'MvnaNl1GEplkpBOBe6nOyw', '2f8o3-FRnq3rwmstUCOZ7A', 'ABzOhfVTcjxwvntaFqHQNw', 'qj2T7GkIVcKCPDFuEqQUrw', 'dRejAgrPIwD6AcykS5Re9g', 'TfW-f8ZtkS5XpIWe9GNZ1g', 'RGEIUdoDtlhlXlkxPKClUg', '24Pyg1ywhjaJDaE9v2tWFQ', 'm66lgElkNdZh4LEDFZ0mNQ', 'PxPJmIAVC7tbEvwQwrmstw', 'LvU4uahiPfogOYL2CmQUDQ', 'Irsdg76tjscSZ2QsvpmYVQ', 'B0xZBB7-oBEiAf_IdqpFWw', '4uDW2Rp9eAPL5BUd5C5-bA', 'TW44bipmQ5fCYbA2D2sajw', 'aGulqXnFmp

In [55]:
file = 'yelp_academic_dataset_business.json'
business_locations = {}
addresses = []
names=[]
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
        if r_line['business_id'] in set(sick_ids):
            business_locations.update({r_line['business_id']:(r_line['full_address']).rsplit('\n', 5)[0].lower()})
            addresses.append(r_line['full_address'].rsplit('\n', 5)[0].lower())
            names.append(r_line['name'].lower())
addresses = np.asarray(addresses)
df_agg_revs['address']=addresses
df_agg_revs['name']=names
#print(business_locations)

In [56]:
df_agg_revs.head()

,business_id,stars,text,votes,address,name
0,0tUlPATGNNR7zq5uzh3nog,"[4, 4, 4, 2, 1, 3, 3, 3, 1, 3, 2, 1, 2, 3, 1, ...","[With 3 stars as ""A-OK"" and 4 stars as ""Yay! ...","[{'useful': 2, 'funny': 2, 'cool': 3}, {'usefu...",1916 village center cir,chianti cafe
1,9RjV5ZT6Np-VsK96_K1KlQ,"[3, 1, 5, 4]",[Ate at this place a few days ago. I ordered ...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",2737 las vegas blvd n,rigos taco
2,994TtJYxB68eSscnCr94qQ,"[5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, 4, 3, 3, 5, ...","[This place is fantastic. The pizza is great,...","[{'useful': 3, 'funny': 0, 'cool': 0}, {'usefu...",2312 e craig rd,mulligans bar & grill
3,fAJYeE4Q6vwVPGkk1MWCoQ,"[2, 3, 1, 3, 4, 5, 5, 2, 5, 2, 4, 4, 4, 4, 3, ...",[Don't let the newly refurbished decor fool yo...,"[{'useful': 0, 'funny': 1, 'cool': 0}, {'usefu...",3100 e lake mead blvd,domino's pizza
4,2mKC1oXdGCl2w8SAZK8fjw,"[4, 4, 4, 1, 3, 4, 5, 5, 3, 5, 4, 3, 3, 4, 4, ...",[Delights Cafe just started serving breakfast ...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",1824 e lake mead boulevard,carl's jr restaurants


In [41]:
for x in business_locations:
    print(x)

C3WAtjq006LVC-EHCt-fRA
4O9Du_JiAvz24b2VSOJNTA
jWiGtY7hTqBjbzN3yyRXbw
IV-1NJnGA3O0ZRBrZEWWaA
V4XYRuhq_614tFFvWK5q6g
jgkT9RA3OCBqgniWCkwLrA
-GF267VCCEZ7es8pnKxRoA
1nO6ffHEwZYFBzEsyw1w7w
FV16IeXJp2W6pnghTz2FAw
0BVxe-S5aZMa0GgIYqDmHQ
8zTes4kUSoM161AD7G6DsQ
w29d_3Ua0_JlOaAk89Sr2w
O9ro4MvZ7LDQC76zLEg7AQ
UQEf2T_XYszW2F_y7rBNLA
fuCKyA_ZTXVEo8BwUaeh1Q
tl-yAtKfAB7wtCWehVH1cQ
aGbjLWzcrnEx2ZmMCFm3EA
mb4B1LdcEkz4V6BCm-LeFw
HXmKSaJa7hXsgeCozQs-uQ
XpdKTCAPOSUj50fTnW94aA
OK-v9HfG53SCFvZ4mn5IYQ
1EF9J1TGjxX5gXB3U8hjvA
gM8YcvjG2G8CTgCarkBAeA
uXJbNkU6cKAooMWTWnl5aA
3Igoz7noWksymLqoMWuzRg
shA3pC1JuAkXl28_LVcp7w
O8BRzdXLBzu6K4Sm32fxng
6ZaA_KOS1wW5PqDPAHH9Qg
FDfK4HygC_P-Idoy_K8PKQ
-CIZh4JOZuOEbSzLVrAIGw
YO-hq425916TWm4Rj2-Y-A
N0sUdCBcA3ScYULAkGOqxA
pAWgUsCkPwJG4Vw75Wk0aw
w8sRfLP8U7uX4OktcSbb_w
CfdCsVkzzM3uJKGRzA0ufA
957AgCsV5hhFLDJ5aw546w
cWFpyN49FRsFrNd3_cb50g
XQvKQDAyE3vLsDdjJBKs4Q
X8P2i2tyhLSzljdliBFBrg
sAbRiVqmxNdjTja_vmnsnA
oy90Dd3UjsyAmj_lxSda5g
8buIr1zBCO7OEcAQSZko7w
TNIR0ht13OSqG-HlkoG6uQ
gPmoQaWDiVn

In [58]:
# create lists for addresses of businesses with violations in order to 
# check for overlap between the Las Vegas Violation set and the yelp set.

violation_addresses = []

for line in df_agg['Address']:
    violation_addresses.append({df_agg[str(line).lower())
violation_addresses[:10]

#overlap_df_revs=df_agg_revs[df_agg_revs['address'].isin(violation_addresses)]
#overlap_df_agg=df_agg[df_agg['Address'].isin(overlap_df_revs['address'])]
#overlap_df_revs=overlap_df_revs.reset_index(drop = True)
#overlap_df_revs

,business_id,stars,text,votes,address,name
0,0aV8mzabKSFylfi0NQGqlQ,"[3, 5, 3, 5, 4, 4, 5, 5, 5, 5, 2, 5, 4, 5, 4, ...",[What a dump. I thought my friends were taking...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",4355 spring mountain rd,cafe noodle & chinese barbeque
1,qpyD1Bg2JbmQebygwd499g,"[4, 4, 5, 1, 5, 1, 4, 5, 1, 1, 1, 5, 4, 1, 3, ...",[This is definitely a hidden jewel. They are a...,"[{'useful': 4, 'funny': 0, 'cool': 1}, {'usefu...",3360 w sahara ave,courtyard deli
2,XGcsblboBZ0Y6VuAfVXxvg,"[4, 5, 5, 5]",[Probably will change to 5 store once they hav...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",4215 spring mountain rd,pho vietnam restaurant
3,c9ENPUTfi5blUTPgkIHu3w,"[4, 5, 4, 3, 4, 4, 3, 4, 4, 4, 4, 5, 3, 5, 5, ...",[I came here with my boyfriend for some late n...,"[{'useful': 3, 'funny': 1, 'cool': 2}, {'usefu...",3886 w sahara ave,china joe's
4,UZVuNXCNe4yA-gRsayw1VA,"[3, 4, 3, 2]","[Eisenhower once said, ""Some people wanted cha...","[{'useful': 2, 'funny': 0, 'cool': 1}, {'usefu...",3111 s valley view blvd,las pupusas restaurant
5,unxjJu6JOxVKwGTVdtaXMQ,"[3, 2, 1, 4, 1, 4, 2, 2, 2, 2, 4, 3, 3, 3, 2, ...",[It is decent for what it is and I think stick...,"[{'useful': 1, 'funny': 1, 'cool': 1}, {'usefu...",4745 spring mountain rd,pho so 1
6,LZklCs8ZGLOFIeQVcoedgQ,"[2, 3, 3, 1, 2, 1, 3, 3, 3, 1, 3, 3, 3, 1, 3, ...","[On my last trip out, I was stuck in traffic o...","[{'useful': 1, 'funny': 3, 'cool': 2}, {'usefu...",4355 spring mountain rd,pho nhu y
7,qCvRV9qcHKftXnT9IkmTqA,"[2, 2, 4, 4, 2, 3, 1, 4, 1, 1, 1, 1, 1, 2, 2]",[I was seriously considering giving this a 1 s...,"[{'useful': 2, 'funny': 0, 'cool': 1}, {'usefu...",3920 spring mountain rd,tofu hut
8,4kb6cLYeuFqK-HCfCu3IcQ,"[1, 1, 3, 5, 1, 1, 1, 1, 1]",[One star because McDonald's is not food... it...,"[{'useful': 0, 'funny': 2, 'cool': 4}, {'usefu...",2101 s decatur blvd,dotty's #7
9,JQKOBllhtI_X5ZjpWSum6A,"[4, 2, 4, 3, 2, 1, 2, 3, 1, 1, 3, 3, 4, 1, 3, 1]","[As far as Las Vegas dim sum restaurants goes,...","[{'useful': 3, 'funny': 2, 'cool': 3}, {'usefu...",2003 s decatur blvd,subway


In [59]:
overlap_df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y


In [31]:
def compare()

911


It looks like there are a total of 911 unique restaurant addresses that have an evaluated sick score greater than 0 and entries in the inspection dataset.

In [32]:
categories = df_agg.columns
df_agg = df_agg.drop(['Inspection Grade'], axis=1)
df_agg = df_agg.drop(['Inspection Demerits'], axis=1)              

In [33]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,PR0110047,CHINA DRAGON/#1 HAWAIIAN BARBEQUE,Restaurant,5905 S EASTERN Ave 105,9,A,12/21/2015 12:00:00 AM,NaN,Compliant,211215292829302955,36.082,-115.12
1,$5 CAFE,PR0017837,AZTEC INN - $5 CAFE,Restaurant,2200 S Las Vegas Blvd,10,A,01/07/2016 12:00:00 AM,NaN,Compliant,202206291029282930,36.1459,-115.156
2,100 DEGREES HOT POT - DELETED 10/15,PR0113718,100 DEGREES HOT POT,Restaurant,5600 SPRING MOUNTAIN Rd B,3,A,04/27/2015 12:00:00 AM,NaN,Compliant,2122911,36.1265,-115.218
3,101 MART SNACK BAR,PR0118514,101 MART,Snack Bar,1021 CONVENTION CENTER Dr,6,A,07/26/2016 12:00:00 AM,NaN,'A' Grade,2152955,36.1322,-115.155
4,168 Market - Bakery,PR0021447,168 Market,Bakery Sales,3459 S Jones Blvd,3,A,07/05/2016 12:00:00 AM,NaN,'A' Grade,217,36.1266,-115.225
5,168 Market - Restaurant,PR0021446,168 Market,Restaurant,3459 S Jones Blvd,0,A,10/14/2015 12:00:00 AM,NaN,Compliant,291029282930,36.1266,-115.225
6,168 Market / Produce Department,PR0021445,168 Market,Produce Market,3459 S Jones Blvd,6,A,07/28/2016 12:00:00 AM,NaN,'A' Grade,2152955,36.1266,-115.225
7,17 SOUTH BOOZE & BITES BAR,PR0109450,Tahiti Village,Bar / Tavern,7200 S Las Vegas Blvd,0,A,05/18/2016 12:00:00 AM,NaN,Compliant,NaN,36.0597,-115.172
8,17 SOUTH BOOZE & BITES GRILL,PR0109449,Tahiti Village,Restaurant,7200 S Las Vegas Blvd,9,A,05/18/2016 12:00:00 AM,NaN,Compliant,21221629292955,36.0597,-115.172
9,1900 ASIAN CUISINE,PR0099062,1900 ASIAN CUISINE,Restaurant,5115 Spring Mountain Rd 103,0,A,11/22/2013 12:00:00 AM,NaN,Compliant,29102927,36.1255,-115.211


In [73]:
df_cat = pd.get_dummies(df_agg['Category Name'], prefix = 'category')
df_grade = pd.get_dummies(df_agg['Current Grade'], prefix = 'grade_cur')
df_perm = pd.get_dummies(df_agg['Permit Status'], prefix = 'permit')
df_res = pd.get_dummies(df_agg['Inspection Result'], prefix = 'result')
df_dems = pd.get_dummies(df_agg['Current Demerits'], prefix = 'demerits')
df_type = pd.get_dummies(df_agg['Inspection Type'], prefix = 'type')
df_new = pd.concat([df_agg, df_cat], axis=1)
df_new = pd.concat([df_new, df_grade], axis=1)
df_new = pd.concat([df_new, df_perm], axis=1)
df_new = pd.concat([df_new, df_res], axis=1)
df_new = pd.concat([df_new, df_dems], axis=1)
df_new = pd.concat([df_new, df_type], axis=1)
df_new = df_new.drop(['Category Name'], axis = 1 )
df_new = df_new.drop(['Current Grade'], axis = 1 )
df_new = df_new.drop(['Permit Status'], axis = 1 )
df_new = df_new.drop(['Inspection Result'], axis = 1)
df_new = df_new.drop(['Current Demerits'], axis = 1)
df_new = df_new.drop(['Inspection Type'], axis = 1)
df_new = df_new.drop(['Date Current'], axis = 1)
df_new = df_new.drop(['Restaurant Name'], axis = 1)
df_new = df_new.drop(['Permit Number'], axis = 1)
df_new = df_new.drop(['Location Name'], axis = 1)
df_new = df_new.drop(['Address'], axis = 1)
df_new.fillna(0)

KeyError: 'Inspection Type'

In [ ]:
df_new

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

X = df_new.loc[:,:].values
y = df_new['grade_cur_X'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, )
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)
print(pred)
test_score = knn.score(X_test,y_test)
print('Accuracy on test data: {}'.format(test_score))


[ 0.  0.  0. ...,  0.  0.  0.]
Accuracy on test data: 0.9997120644975526


In [62]:
X = df_new.loc[:,:].values
y = df_new['grade_cur_A'].values
X_train, X_test, y_train, y_test = train_test_split(X, y)
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)
print(pred)
test_score = knn.score(X_test,y_test)
print('Accuracy on test data: {}'.format(test_score))

[ 1.  1.  1. ...,  1.  1.  1.]
Accuracy on test data: 0.9979844514828679
